In [2]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value
import csv
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("Final_data.csv")

In [4]:
df.shape

(507, 6)

In [5]:
df.head(4)

,DMUs,Investment_inf,Maintenance_inf,Transport_inf,Value_added,CO2_Emissions
0,A,99.94,35.59,59.51,5.62,98.07
1,B,99.83,37.38,59.37,5.61,97.80
2,C,99.42,38.88,63.05,5.68,97.76
3,D,99.65,39.20,62.36,5.61,97.87


In [6]:
K = df.iloc[:, 0]
I = df.iloc[:, [1,2,3]]
J = df.iloc[:, [4,5]]

In [7]:
# Parameters building
X = { 
    i: {
        k: 0 for k in K
    } for i in I
}
Y = { 
    j: {
        k: 0 for k in K
    } for j in J
}
# Import CSV data
with open('Final_data.csv', newline='') as csvfile:
    rows = csv.DictReader(csvfile)
    k = 0
    for row in rows:
        for i in I:
            X[i][K[k]] = float(row[i]) 
        for j in J:
            Y[j][K[k]] = float(row[j])
        k += 1

In [8]:
# CRS_DEA_Model        
def getOverallEfficiency(r):

    # Model Building
    model = LpProblem('CRS_model', LpMinimize) # Build model
    
    # Decision variables Building
    theta_r = LpVariable(f'theta_r')
    lambda_k = LpVariable.dicts(f'lambda_k', lowBound=0, indices=K)
    
    # Objective Function setting
    model += theta_r   #Dual formulation
    
    # Constraints setting
    for i in I:
        model += lpSum([
                lambda_k[k] * X[i][k]
            for k in K]) <= theta_r * float(X[i][K[r]])
    for j in J:
        model += lpSum([
                lambda_k[k] * Y[j][k]
            for k in K]) >= float(Y[j][K[r]])
    
    # Model solving
    model.solve()
    
    return f'{K[r]};{round(value(model.objective), 3)}\n', value(model.objective)

In [9]:
# VRS_DEA_Model
def getTechnicalEfficiency(r):

    # Model Building 
    model = LpProblem('VRS_model', LpMinimize) # Build model
    
    # Decision variables Building 
    theta_r = LpVariable(f'theta_r')
    lambda_k = LpVariable.dicts(f'lambda_k', lowBound=0, indices=K)
    
    # Objective Function setting
    model += theta_r
    
    # Constraints setting
    for i in I:
        model += lpSum([
                lambda_k[k] * X[i][k]
            for k in K]) <= theta_r * float(X[i][K[r]])
    for j in J:
        model += lpSum([
                lambda_k[k] * Y[j][k]
            for k in K]) >= float(Y[j][K[r]])
    model += lpSum([ lambda_k[k] for k in K]) == 1
    
    # model solving 
    model.solve()  
    
    return f'{K[r]};{round(value(model.objective), 3)}\n', value(model.objective)

In [10]:
OE_outputText = 'These are OE of all DMUs\n-------------\n'
TE_outputText = 'These are TE of all DMUs\n-------------\n'
SE_outputText = 'These are SE of all DMUs\n-------------\n'

for k in range(len(K)):
        OE_text, OE_val = getOverallEfficiency(k)
        TE_text, TE_val = getTechnicalEfficiency(k)
        OE_outputText += OE_text
        TE_outputText += TE_text
        SE_outputText += f'{K[k]};{round(OE_val / TE_val, 3)}\n'
print(OE_outputText)
print(TE_outputText)
print(SE_outputText)

These are OE of all DMUs
-------------
A;0.843
B;0.839
C;0.84
D;0.838
E;0.834
F;0.814
G;0.813
H;0.811
I;0.813
J;0.805
K;0.811
L;0.813
M;0.815
N;0.809
O;0.824
P;0.807
Q;0.807
R;0.824
S;0.81
T;0.812
U;0.924
V;0.925
W;0.928
X;0.936
Y;0.948
Z;0.979
AA;0.964
AB;0.925
AC;0.896
AD;1.0
AE;0.962
AF;1.0
AG;0.931
AH;1.0
AI;0.923
AJ;0.945
AK;0.955
AL;0.963
AM;1.0
AN;1.0
AO;0.769
AP;0.772
AQ;0.887
AR;0.895
AS;0.911
AT;0.92
AU;0.894
AV;0.883
AW;0.896
AX;0.904
AY;0.883
AZ;0.861
BA;0.857
BB;0.851
BC;0.851
BD;0.85
BE;0.852
BF;0.848
BG;0.846
BH;0.86
BI;0.857
BJ;0.851
BK;0.834
BL;0.837
BM;0.844
BN;0.841
BO;0.837
BP;0.842
BQ;0.824
BR;0.817
BS;0.832
BT;0.822
BU;0.821
BV;0.826
BW;0.839
BX;0.83
BY;0.825
BZ;0.822
CA;0.83
CB;0.957
CC;0.946
CD;1.0
CE;0.961
CF;0.967
CG;0.942
CH;0.932
CI;0.943
CJ;0.966
CK;0.974
CL;0.979
CM;0.979
CN;0.958
CO;0.961
CP;0.94
CQ;0.949
CR;0.958
CS;0.95
CT;0.949
CU;0.956
CV;0.816
CW;0.81
CX;0.809
CY;0.806
CZ;0.799
DA;0.802
DB;0.799
DC;0.797
DD;0.795
DE;0.794
DF;0.864
DG;0.873
DH;0.87
DI